In [1]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer

device = 'cuda'
tokenizer: T5Tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

model: T5ForConditionalGeneration = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base").to(device)

d_ff = 2048,
d_kv = 64,
d_model = 768,
decoder_start_token_id = 0,
dense_act_fn = 'gelu_new',
dropout_rate = 0.1,
eos_token_id = 1,
feed_forward_proj = 'gated-gelu',
initializer_factor = 1.0,
is_encoder_decoder = True,
is_gated_act = True,
layer_norm_epsilon = 1e-06,
model_type = 't5',
n_positions = 512,
num_decoder_layers = 12,
num_heads = 12,
num_layers = 12,
output_past = True,
pad_token_id = 0,
relative_attention_max_distance = 128,
relative_attention_num_buckets = 32,
tie_word_embeddings = False,
use_cache = True,
vocab_size = 32128,
task_specific_params = {
    "summarization": {
        "early_stopping": True,
        "length_penalty": 2.0,
        "max_length": 200,
        "min_length": 30,
        "no_repeat_ngram_size": 3,
        "num_beams": 4,
        "prefix": "summarize: "
    },
    "translation_en_to_de": {
        "early_stopping": True,
        "max_length": 300,
        "num_beams": 4,
        "prefix": "translate English to German: "
    },
    "translation_en_to_fr": {
        "early_stopping": True,
        "max_length": 300,
        "num_beams": 4,
        "prefix": "translate English to French: "
    },
    "translation_en_to_ro": {
        "early_stopping": True,
        "max_length": 300,
        "num_beams": 4,
        "prefix": "translate English to Romanian: "
    }
}

# console = Console(record=True)



https://huggingface.co/google/flan-t5-base/resolve/main/tokenizer_config.json
https://huggingface.co/google/flan-t5-base/resolve/main/config.json
https://huggingface.co/google/flan-t5-base/resolve/main/generation_config.json


In [ ]:
context: str = """we have 5 glasses and they are each one cost  $100 and they are available in 5 different colors"""

In [7]:
input_text = 'how many colors this glasses have????????'
inp = 'context: ' + context + " question: " + input_text + ' ?'
# inp = 'Translate English to German: ' + input_text
input_ids = tokenizer(inp, return_tensors="pt").input_ids.to(device)

outputs = model.generate(input_ids, max_length=800)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

5


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

https://huggingface.co/google/flan-t5-base/resolve/main/tokenizer_config.json
https://huggingface.co/google/flan-t5-base/resolve/main/config.json
https://huggingface.co/google/flan-t5-base/resolve/main/generation_config.json


In [3]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [4]:
model.state_dict()

OrderedDict([('shared.weight',
              tensor([[-4.0805e-01,  1.2728e+00, -1.3392e-01,  ..., -8.0138e-01,
                        3.9240e+00,  9.7175e-03],
                      [ 1.5827e+01,  7.1912e+00,  1.5141e+01,  ...,  5.4508e+00,
                       -2.5928e+01,  1.1496e+01],
                      [-1.8447e+00, -8.8907e-01, -1.2932e+01,  ...,  4.1089e+00,
                        5.0571e+00, -3.7695e+00],
                      ...,
                      [ 5.6641e-01, -1.3594e+00, -1.1953e+00,  ...,  1.4648e-01,
                       -5.0781e-01, -1.2109e+00],
                      [-8.9062e-01,  8.1641e-01,  6.3672e-01,  ..., -1.2012e-01,
                        9.4922e-01, -1.5938e+00],
                      [-8.3203e-01,  1.8281e+00,  1.3770e-01,  ..., -2.1094e-01,
                       -1.1875e+00, -5.1172e-01]])),
             ('encoder.embed_tokens.weight',
              tensor([[-4.0805e-01,  1.2728e+00, -1.3392e-01,  ..., -8.0138e-01,
                        3.9

In [16]:
from modules.modeling_LLmPU import LLmPUForConditionalGeneration

In [17]:
model.config

T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_

In [18]:
fk_model = LLmPUForConditionalGeneration(config=model.config)

In [19]:
fk_model

LLmPUForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): LLmPUStack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): LLmPUBlock(
        (layer): ModuleList(
          (0): LLmPULayerSelfAttention(
            (SelfAttention): LLmPUAttention(
              (relative_attention_bias): Embedding(32, 12)
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
            )
            (layer_norm): LLmPULayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): LLmPULayerFF(
            (DenseReluDense): LLmPUDenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=Fa

In [20]:
fk_model.load_state_dict(model.state_dict())

<All keys matched successfully>

In [21]:
fk_model.generate()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Erfun\AppData\Local\Temp\ipykernel_176\2804050362.py:1 in <module>                      │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\Erfun\\AppData\\Local\\Temp\\ipykernel_176\\2804050362.py'                           │
│                                                                                                  │
│ E:\Conda3\lib\site-packages\torch\nn\modules\module.py:1591 in __getattr__                       │
│                                                                                                  │
│   1588 │   │   │   modules = self.__dict__['_modules']                                           │
│   1589 │   │   │   if name in modules:                                                           │
│   1590 │   │   │   │   return modules[name]                                                      │
│ ❱ 1591 │   │   raise AttributeError("'{}' object has no attribute '{}'".format(                  │
│   1592 │   │   │   type(self).__name__, name))                                                   │
│   1593 │                                                                                         │
│   1594 │   def __setattr__(self, name: str, value: Union[Tensor, 'Module']) -> None:             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'LLmPUForConditionalGeneration' object has no attribute 'generate'

In [22]:
model.generate()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Erfun\AppData\Local\Temp\ipykernel_176\336120273.py:1 in <module>                       │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ 'C:\\Users\\Erfun\\AppData\\Local\\Temp\\ipykernel_176\\336120273.py'                            │
│                                                                                                  │
│ E:\Conda3\lib\site-packages\torch\autograd\grad_mode.py:34 in decorate_context                   │
│                                                                                                  │
│    31 │   │   @functools.wraps(func)                                                             │
│    32 │   │   def decorate_context(*args, **kwargs):                                             │
│    33 │   │   │   with self.clone():                                                             │
│ ❱  34 │   │   │   │   return func(*args, **kwargs)                                               │
│    35 │   │   return cast(F, decorate_context)                                                   │
│    36 │                                                                                          │
│    37 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ E:\Conda3\lib\site-packages\transformers\generation\utils.py:1220 in generate                    │
│                                                                                                  │
│   1217 │   │   # model_input_name is defined if model-specific keyword input is passed           │
│   1218 │   │   # otherwise model_input_name is None                                              │
│   1219 │   │   # all model-specific keyword inputs are removed from `model_kwargs`               │
│ ❱ 1220 │   │   inputs_tensor, model_input_name, model_kwargs = self._prepare_model_inputs(       │
│   1221 │   │   │   inputs, generation_config.bos_token_id, model_kwargs                          │
│   1222 │   │   )                                                                                 │
│   1223 │   │   batch_size = inputs_tensor.shape[0]                                               │
│                                                                                                  │
│ E:\Conda3\lib\site-packages\transformers\generation\utils.py:543 in _prepare_model_inputs        │
│                                                                                                  │
│    540 │   │                                                                                     │
│    541 │   │   # 5. if `inputs` is still None, try to create `input_ids` from BOS token          │
│    542 │   │   if inputs is None:                                                                │
│ ❱  543 │   │   │   inputs = self._prepare_input_ids_for_generation(bos_token_id, model_kwargs.g  │
│    544 │   │                                                                                     │
│    545 │   │   return inputs, input_name, model_kwargs                                           │
│    546                                                                                           │
│                                                                                                  │
│ E:\Conda3\lib\site-packages\transformers\generation\utils.py:578 in                              │
│ _prepare_input_ids_for_generation                                                                │
│                                                                                                  │
│    575 │   │   │   return torch.ones(shape, dtype=torch.lon